# 2. Stream Consumption & Writing to Cassandra 

## 2.0 Cassandra workflow

In [1]:
import pyspark
import time
import os


os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 pyspark-shell'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row


conf = SparkConf() \
    .setAppName("CassandraWriter") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "127.0.0.1")
    
spark = SparkContext(conf=conf)
sqlContext=SQLContext(spark)

data_path = "../Data"

In [3]:
from pyspark.sql import DataFrame
import functools


# Define helper function to concatenate streaming DataFrames
def unionAll(df_list):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), df_list)


dfs = []
for year in range(1995, 2009):
    print(year)
    for month in range(1, 13):
        csv_path = data_path + "/" + str(year) + "/" + str(month)

#         Use of Spark static DataFrames because aggregation operations (e.g., JOINS) are not supported for streaming datasets 
        dfs.append(sqlContext.read.options(header='True', inferSchema='True', delimiter=',').csv(data_path + "/" + str(year) + "/" + str(month) + "/" + str(month) + ".csv"))
        
df = unionAll(dfs)

1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008


In [4]:
df.createOrReplaceTempView("flights")

(Optional) To save data to Cassandra, please run the following cell:

In [5]:
# # Cassandra requires a unique primary key, that we add here + column names must be in lowercase format
# from pyspark.sql.functions import monotonically_increasing_id 


# df_index = df.toDF(*[col.lower() for col in df.columns]).withColumn("id", monotonically_increasing_id())
# df_index.createOrReplaceTempView("flights")
# df_index.write.format("org.apache.spark.sql.cassandra").mode('overwrite').options(table="flights", keyspace="task2").save()

## 2.1 For each airport X, rank the top-10 carriers in decreasing order of on-time departure performance from X.
Let us first populate the "task2.flights_2_1" table in Cassandra:

In [6]:
table_2_1 = sqlContext.sql("SELECT * FROM(\
                        SELECT \
                        *,\
                        rank() OVER (PARTITION BY origin ORDER BY avg_delay, carrier) AS rank\
                        FROM (\
                        SELECT \
                           ORIGIN AS origin,\
                           OP_UNIQUE_CARRIER AS carrier,\
                           round(avg(DEP_DELAY), 3) AS avg_delay\
                        FROM flights\
                        WHERE CANCELLED=0 AND DEP_DELAY IS NOT NULL\
                        GROUP BY ORIGIN, OP_UNIQUE_CARRIER))\
                        WHERE rank <= 10")

table_2_1.createOrReplaceTempView("flights_2_1")
table_2_1.show()

+------+-------+---------+----+
|origin|carrier|avg_delay|rank|
+------+-------+---------+----+
|   BGM|     DH|    1.669|   1|
|   BGM|     OH|    2.068|   2|
|   BGM|     US|    4.821|   3|
|   BGM|     9E|    8.388|   4|
|   BGM|     EV|   13.624|   5|
|   DLG|     AS|   12.105|   1|
|   INL|     9E|    6.169|   1|
|   PSE|     CO|     0.97|   1|
|   PSE|     B6|     2.03|   2|
|   MSY|     HP|    2.373|   1|
|   MSY|     NW|    2.514|   2|
|   MSY|     XE|    4.222|   3|
|   MSY|     F9|    4.464|   4|
|   MSY|     9E|      5.2|   5|
|   MSY|     DL|    5.427|   6|
|   MSY|     US|    5.443|   7|
|   MSY|     CO|    6.027|   8|
|   MSY|     OO|     6.54|   9|
|   MSY|     AA|    6.739|  10|
|   GEG|     US|    2.362|   1|
+------+-------+---------+----+
only showing top 20 rows



(Optional) To save data to Cassandra, please run the following cell:

In [ ]:
# table_2_1.write.format("org.apache.spark.sql.cassandra").mode('overwrite').options(table="flights_2_1", keyspace="task2").save()

(Optional) To extract data from Cassandra, please run the following cell:

In [ ]:
# query_2_1 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="flights_2_1", keyspace="task2").load()
# query_2_1.createOrReplaceTempView("flights_2_1")

And we run the query:

In [20]:
sqlContext.sql("SELECT * FROM flights_2_1 WHERE origin IN ('SRQ', 'CMH', 'JFK', 'SEA', 'BOS')").show(50)

+------+-------+---------+----+
|origin|carrier|avg_delay|rank|
+------+-------+---------+----+
|   CMH|     FL|    1.948|   1|
|   CMH|     DH|    3.491|   2|
|   CMH|     AA|     3.53|   3|
|   CMH|     DL|    4.535|   4|
|   CMH|     NW|    4.969|   5|
|   CMH|     US|     5.66|   6|
|   CMH|     YV|    7.961|   7|
|   CMH|     TW|     8.03|   8|
|   CMH|     9E|    8.326|   9|
|   CMH|     WN|     8.37|  10|
|   SRQ|     TZ|   -0.382|   1|
|   SRQ|     XE|     1.49|   2|
|   SRQ|     YV|    3.426|   3|
|   SRQ|     US|    3.489|   4|
|   SRQ|     DL|     5.31|   5|
|   SRQ|     MQ|    5.351|   6|
|   SRQ|     TW|    5.967|   7|
|   SRQ|     FL|    6.061|   8|
|   SRQ|     NW|    6.443|   9|
|   SRQ|     CO|    9.286|  10|
|   BOS|     TZ|    3.064|   1|
|   BOS|     NW|    6.951|   2|
|   BOS|     DL|    7.162|   3|
|   BOS|     EV|    7.208|   4|
|   BOS|     US|    7.941|   5|
|   BOS|     XE|    8.987|   6|
|   BOS|     AA|    9.178|   7|
|   BOS|     UA|    9.577|   8|
|   BOS|

## 2.2 For each airport X, rank the top-10 airports in decreasing order of on-time departure performance from X.
Let us first populate the "task2.flights_2_2" table in Cassandra:

In [8]:
table_2_2 = sqlContext.sql("SELECT * FROM(\
                        SELECT \
                        *,\
                        rank() OVER (PARTITION BY origin ORDER BY avg_delay, destination) AS rank\
                        FROM (\
                        SELECT \
                           ORIGIN AS origin,\
                           DEST AS destination,\
                           round(avg(DEP_DELAY), 3) AS avg_delay\
                        FROM flights\
                        WHERE CANCELLED=0 AND DEP_DELAY IS NOT NULL\
                        GROUP BY ORIGIN, DEST))\
                        WHERE rank <= 10")

table_2_2.createOrReplaceTempView("flights_2_2")
table_2_2.show()

+------+-----------+---------+----+
|origin|destination|avg_delay|rank|
+------+-----------+---------+----+
|   BGM|        IAD|    1.669|   1|
|   BGM|        CVG|    2.026|   2|
|   BGM|        PHL|    4.576|   3|
|   BGM|        PIT|    4.829|   4|
|   BGM|        JFK|    7.824|   5|
|   BGM|        DTW|    8.388|   6|
|   BGM|        ATL|   13.624|   7|
|   DLG|        ANC|   11.918|   1|
|   DLG|        AKN|   17.375|   2|
|   DLG|        ADQ|   30.923|   3|
|   INL|        MSP|    6.169|   1|
|   PSE|        JFK|    0.796|   1|
|   PSE|        EWR|     0.97|   2|
|   PSE|        MCO|    4.038|   3|
|   PSE|        FLL|    6.374|   4|
|   MSY|        CAE|     -5.0|   1|
|   MSY|        TWF|     -3.0|   2|
|   MSY|        JAN|      0.0|   3|
|   MSY|        CVG|    1.152|   4|
|   MSY|        MEM|    1.713|   5|
+------+-----------+---------+----+
only showing top 20 rows



(Optional) To save data to Cassandra, please run the following cell:

In [9]:
# table_2_2.write.format("org.apache.spark.sql.cassandra").mode('overwrite').options(table="flights_2_2", keyspace="task2").save()

(Optional) To extract data from Cassandra, please run the following cell:

In [10]:
# query_2_2 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="flights_2_2", keyspace="task2").load()
# query_2_2.createOrReplaceTempView("flights_2_2")

And we run the query:

In [21]:
sqlContext.sql("SELECT * FROM flights_2_2 WHERE origin IN ('SRQ', 'CMH', 'JFK', 'SEA', 'BOS')").show(50)

+------+-----------+---------+----+
|origin|destination|avg_delay|rank|
+------+-----------+---------+----+
|   CMH|        AUS|     -5.0|   1|
|   CMH|        OMA|     -5.0|   2|
|   CMH|        SYR|     -5.0|   3|
|   CMH|        CLE|    0.526|   4|
|   CMH|        MSN|      1.0|   5|
|   CMH|        SLC|    3.555|   6|
|   CMH|        CLT|    3.667|   7|
|   CMH|        IAD|    4.158|   8|
|   CMH|        MEM|    4.295|   9|
|   CMH|        IAH|    4.381|  10|
|   SRQ|        IAH|   -0.688|   1|
|   SRQ|        TPA|    -0.56|   2|
|   SRQ|        EYW|      0.0|   3|
|   SRQ|        DFW|    1.858|   4|
|   SRQ|        FLL|      2.0|   5|
|   SRQ|        MCO|    2.066|   6|
|   SRQ|        RSW|    2.199|   7|
|   SRQ|        CLE|    2.462|   8|
|   SRQ|        MDW|    2.638|   9|
|   SRQ|        CLT|    2.915|  10|
|   BOS|        SWF|     -5.0|   1|
|   BOS|        ONT|     -3.0|   2|
|   BOS|        GGG|      1.0|   3|
|   BOS|        AUS|    1.324|   4|
|   BOS|        LGA|    2.74

## 2.3 For each source-destination pair X-Y, rank the top-10 carriers in decreasing order of on-time arrival performance at Y from X.
Let us first populate the "task2.flights_2_3" table in Cassandra:

In [12]:
table_2_3 = sqlContext.sql("SELECT * FROM(\
                        SELECT \
                        *,\
                        rank() OVER (PARTITION BY origin, destination ORDER BY avg_delay, carrier) AS rank\
                        FROM (\
                        SELECT \
                           ORIGIN AS origin,\
                           DEST AS destination,\
                           round(avg(ARR_DELAY), 3) AS avg_delay,\
                           OP_UNIQUE_CARRIER AS carrier\
                        FROM flights\
                        WHERE CANCELLED=0 AND ARR_DELAY IS NOT NULL\
                        GROUP BY ORIGIN, DEST, OP_UNIQUE_CARRIER))\
                        WHERE rank <= 10")

table_2_3.createOrReplaceTempView("flights_2_3")
table_2_3.show()

+------+-----------+---------+-------+----+
|origin|destination|avg_delay|carrier|rank|
+------+-----------+---------+-------+----+
|   ADK|        AKN|   23.732|     AS|   1|
|   ATL|        GSP|    0.082|     CO|   1|
|   ATL|        GSP|    5.041|     OO|   2|
|   ATL|        GSP|    6.459|     DL|   3|
|   ATL|        GSP|    8.677|     9E|   4|
|   ATL|        GSP|    8.793|     EV|   5|
|   ATL|        GSP|   13.665|     OH|   6|
|   AVP|        JFK|   49.882|     OH|   1|
|   BDL|        SLC|   -3.759|     DL|   1|
|   BFL|        SAN|   -3.599|     XE|   1|
|   BOI|        SBA|    3.227|     OO|   1|
|   BQN|        MCO|    0.256|     B6|   1|
|   CLE|        SJU|   -0.912|     CO|   1|
|   DSM|        EWR|   26.938|     XE|   1|
|   EWR|        STT|    2.425|     CO|   1|
|   FSD|        ATL|    6.559|     EV|   1|
|   FSD|        ATL|    9.582|     OH|   2|
|   GRR|        PIT|    2.994|     US|   1|
|   HTS|        MCN|    122.0|     EV|   1|
|   LAS|        LIT|    1.229|  

(Optional) To save data to Cassandra, please run the following cell:

In [13]:
# table_2_3.write.format("org.apache.spark.sql.cassandra").mode('overwrite').options(table="flights_2_3", keyspace="task2").save()

(Optional) To extract data from Cassandra, please run the following cell:

In [14]:
# query_2_3 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="flights_2_3", keyspace="task2").load()
# query_2_3.createOrReplaceTempView("flights_2_3")

And we run the query:

In [23]:
sqlContext.sql("SELECT * FROM flights_2_3 \
                WHERE (origin='LGA' AND destination='BOS') \
                OR (origin='BOS' AND destination='LGA')\
                OR (origin='OKC' AND destination='DFW')\
                OR (origin='MSP' AND destination='ATL')\
                ORDER BY origin, rank ASC").show(40)

+------+-----------+---------+-------+----+
|origin|destination|avg_delay|carrier|rank|
+------+-----------+---------+-------+----+
|   BOS|        LGA|    0.663|     US|   1|
|   BOS|        LGA|     2.16|     DL|   2|
|   BOS|        LGA|   12.482|     MQ|   3|
|   BOS|        LGA|     25.6|     AA|   4|
|   BOS|        LGA|   30.448|     OH|   5|
|   BOS|        LGA|    133.0|     TZ|   6|
|   LGA|        BOS|   -3.168|     US|   1|
|   LGA|        BOS|    1.167|     DL|   2|
|   LGA|        BOS|    9.471|     MQ|   3|
|   LGA|        BOS|   27.985|     OH|   4|
|   LGA|        BOS|     40.0|     AA|   5|
|   MSP|        ATL|    5.378|     OO|   1|
|   MSP|        ATL|    6.021|     DL|   2|
|   MSP|        ATL|    6.398|     FL|   3|
|   MSP|        ATL|     7.74|     NW|   4|
|   MSP|        ATL|    8.352|     OH|   5|
|   MSP|        ATL|   10.287|     EV|   6|
|   OKC|        DFW|    1.359|     EV|   1|
|   OKC|        DFW|    4.049|     AA|   2|
|   OKC|        DFW|     4.71|  

## 2.4 For each source-destination pair X-Y, determine the mean arrival delay (in minutes) for a flight from X to Y.
Let us first populate the "task2.flights_2_4" table in Cassandra:

In [16]:
table_2_4 = sqlContext.sql("SELECT\
                              ORIGIN AS origin,\
                              DEST AS destination,\
                              round(avg(ARR_DELAY), 3) AS avg_delay\
                           FROM flights\
                           WHERE CANCELLED=0\
                           GROUP BY ORIGIN, DEST")

table_2_4.createOrReplaceTempView("flights_2_4")
table_2_4.show()

+------+-----------+---------+
|origin|destination|avg_delay|
+------+-----------+---------+
|   PHL|        MCO|   10.728|
|   MCI|        MKE|    2.011|
|   GRR|        PIT|    2.994|
|   EWR|        STT|    2.425|
|   SMF|        BUR|    6.196|
|   ATL|        GSP|    6.933|
|   ORD|        PDX|   12.276|
|   PBI|        DCA|    4.683|
|   SNA|        PHX|    5.619|
|   MCI|        IAH|    3.875|
|   CLE|        SJU|   -0.912|
|   LAS|        LIT|    1.229|
|   LAX|        OXR|    1.123|
|   BOI|        SBA|    3.227|
|   ROC|        CLE|   13.145|
|   DSM|        EWR|   26.938|
|   ADK|        AKN|   23.732|
|   LNK|        OMA|    131.5|
|   MLI|        MCO|    5.942|
|   HTS|        MCN|    122.0|
+------+-----------+---------+
only showing top 20 rows



(Optional) To save data to Cassandra, please run the following cell:

In [17]:
# table_2_4.write.format("org.apache.spark.sql.cassandra").mode('overwrite').options(table="flights_2_4", keyspace="task2").save()

(Optional) To extract data from Cassandra, please run the following cell:

In [18]:
# query_2_4 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="flights_2_4", keyspace="task2").load()
# query_2_4.createOrReplaceTempView("flights_2_4")

And we run the query:

In [19]:
sqlContext.sql("SELECT * FROM flights_2_4 \
                WHERE (origin='LGA' AND destination='BOS') \
                OR (origin='BOS' AND destination='LGA')\
                OR (origin='OKC' AND destination='DFW')\
                OR (origin='MSP' AND destination='ATL')\
                ORDER BY origin ASC").show()

+------+-----------+---------+
|origin|destination|avg_delay|
+------+-----------+---------+
|   BOS|        LGA|    3.112|
|   LGA|        BOS|    0.952|
|   MSP|        ATL|     6.95|
|   OKC|        DFW|    4.372|
+------+-----------+---------+

